In [1]:
import pandas as pd
import numpy as np
import os, sys
import glob

In [2]:
os.remove('./generation_forecast/.DS_Store')


FileNotFoundError: [Errno 2] No such file or directory: './generation_forecast/.DS_Store'

In [113]:
import glob

solaire_files = glob.glob('./generation_forecast/*Solaire*.csv')
eolien_files = glob.glob('./generation_forecast/*Eolien*.csv')
print(solaire_files)

['./generation_forecast/PrevisionSolaire_2020.xls.csv', './generation_forecast/PrevisionSolaire_2021.xls.csv', './generation_forecast/PrevisionSolaire_2017.xls.csv', './generation_forecast/PrevisionSolaire_2023.xls.csv', './generation_forecast/PrevisionSolaire_2019.xls.csv', './generation_forecast/PrevisionSolaire_2018.xls.csv', './generation_forecast/PrevisionSolaire_2022.xls.csv']


In [127]:
files = [solaire_files, eolien_files]
df_list = []
for i, file in enumerate(solaire_files):

    df = pd.read_csv(file, encoding='latin1', sep='\t')
    df_list.append(df)


In [129]:
solaire_files

['./generation_forecast/PrevisionSolaire_2020.xls.csv',
 './generation_forecast/PrevisionSolaire_2021.xls.csv',
 './generation_forecast/PrevisionSolaire_2017.xls.csv',
 './generation_forecast/PrevisionSolaire_2023.xls.csv',
 './generation_forecast/PrevisionSolaire_2019.xls.csv',
 './generation_forecast/PrevisionSolaire_2018.xls.csv',
 './generation_forecast/PrevisionSolaire_2022.xls.csv']

In [153]:

df_solaire = pd.concat(df_list)

#df_solaire = df_solaire.reset_index(inplace=False).drop('index', axis=1)

mask = df_solaire['Heure'].isna()
df_solaire=df_solaire.loc[~mask]

df_solaire['Heure'] = df_solaire['Heure'].apply(lambda x: "0" + x if len(x) == 4 else x)

df_solaire['Date_Heure_France'] = df_solaire['Date'] + ' ' + df_solaire['Heure']

df_solaire['Date_Heure_France'] = pd.to_datetime(df_solaire['Date_Heure_France'], format = "%d/%m/%Y %H:%M")

df_solaire = df_solaire.reset_index(drop=True)
df_solaire = df_solaire.sort_values(by='Date_Heure_France', axis = 0, ascending=True)

df_solaire = df_solaire[['Date_Heure_France', 'Prévision J-1']]


date_debut = df_solaire['Date_Heure_France'].iloc[0]#.tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_fin = df_solaire['Date_Heure_France'].iloc[-1]#.tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_range = pd.date_range(date_debut, date_fin , freq='1H')
df_solaire = df_solaire.set_index('Date_Heure_France')
here = date_range.isin(df_solaire.index)
holes= date_range[~here]

for i, idx in enumerate(holes):

    df_solaire.loc[idx, 'Prévision J-1'] = 0

df_solaire.loc[df_solaire.index.year == 2023]
date_debut = df_solaire.index[0].tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_range = pd.date_range(date_debut, periods = len(df_solaire), freq='1H')
df_solaire.index = date_range

df_solaire.index = df_solaire.index.rename('Date UTC')

df_solaire['Prévision J-1'].str.startswith('-') == True

df_solaire.loc[df_solaire['Prévision J-1'].str.startswith('-') == True] = 0
df_solaire['Prévision J-1'] = df_solaire['Prévision J-1'].astype(float)

df_solaire.sample(24)

,Prévision J-1
Date UTC,
2017-11-15 02:00:00,0.00
2021-11-17 03:00:00,0.00
2017-02-23 18:00:00,0.00
2017-03-26 22:00:00,0.00
2020-06-14 20:00:00,210.12
2018-07-01 22:00:00,0.00
2017-09-14 16:00:00,2300.86
2018-01-05 08:00:00,119.10
2018-01-26 09:00:00,1193.48


In [154]:

df_eolien = pd.concat(df_list)

#df_eolien = df_solaire.reset_index(inplace=False).drop('index', axis=1)

mask = df_eolien['Heure'].isna()
df_eolien=df_solaire.loc[~mask]

df_eolien['Heure'] = df_solaire['Heure'].apply(lambda x: "0" + x if len(x) == 4 else x)

df_eolien['Date_Heure_France'] = df_solaire['Date'] + ' ' + df_solaire['Heure']

df_eolien['Date_Heure_France'] = pd.to_datetime(df_solaire['Date_Heure_France'], format = "%d/%m/%Y %H:%M")

df_eolien = df_solaire.reset_index(drop=True)
df_eolien = df_solaire.sort_values(by='Date_Heure_France', axis = 0, ascending=True)

df_eolien = df_solaire[['Date_Heure_France', 'Prévision J-1']]


date_debut = df_eolien['Date_Heure_France'].iloc[0]#.tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_fin = df_eolien['Date_Heure_France'].iloc[-1]#.tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_range = pd.date_range(date_debut, date_fin , freq='1H')
df_eolien = df_solaire.set_index('Date_Heure_France')
here = date_range.isin(df_eolien.index)
holes= date_range[~here]

for i, idx in enumerate(holes):

    df_eolien.loc[idx, 'Prévision J-1'] = 0

df_eolien.loc[df_solaire.index.year == 2023]
date_debut = df_eolien.index[0].tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_range = pd.date_range(date_debut, periods = len(df_eolien), freq='1H')
df_eolien.index = date_range

df_eolien.index = df_solaire.index.rename('Date UTC')

df_eolien['Prévision J-1'].str.startswith('-') == True

df_eolien.loc[df_solaire['Prévision J-1'].str.startswith('-') == True] = 0
df_eolien['Prévision J-1'] = df_solaire['Prévision J-1'].astype(float)

df_eolien.sample(24)

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [229]:
date_debut = df_solaire['Date_Heure_France'].iloc[0]#.tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_fin = df_solaire['Date_Heure_France'].iloc[-1]#.tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_range = pd.date_range(date_debut, date_fin , freq='1H')

here = df_solaire.index.isin(date_range)
df_solaire.index[~here]

here = date_range.isin(df_solaire.index)
holes= date_range[~here]

for idx in holes:
    df_solaire.loc[idx, 'Prévision J-1'] = None
    df_solaire.loc[idx, 'Date_Heure_France'] = idx



60696


In [231]:
df_solaire

,Date_Heure_France,Prévision J-1
Date UTC,,
2017-01-01 00:00:00,2017-01-01 00:00:00,0.0
2017-01-01 01:00:00,2017-01-01 01:00:00,0.0
2017-01-01 02:00:00,2017-01-01 02:00:00,0.0
2017-01-01 03:00:00,2017-01-01 03:00:00,0.0
2017-01-01 04:00:00,2017-01-01 04:00:00,0.0
...,...,...
2023-11-30 19:00:00,2023-11-30 19:00:00,NaN
2023-11-30 20:00:00,2023-11-30 20:00:00,NaN
2023-11-30 21:00:00,2023-11-30 21:00:00,NaN


In [233]:
df_solaire

,Date_Heure_France,Prévision J-1
Date UTC,,
2017-01-01 00:00:00,2017-01-01 00:00:00,0.0
2017-01-01 01:00:00,2017-01-01 01:00:00,0.0
2017-01-01 02:00:00,2017-01-01 02:00:00,0.0
2017-01-01 03:00:00,2017-01-01 03:00:00,0.0
2017-01-01 04:00:00,2017-01-01 04:00:00,0.0
...,...,...
2023-11-30 19:00:00,2023-11-30 19:00:00,NaN
2023-11-30 20:00:00,2023-11-30 20:00:00,NaN
2023-11-30 21:00:00,2023-11-30 21:00:00,NaN


In [232]:
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", None)
df_solaire.describe()

,Prévision J-1
count,54681.000000
mean,1464.660966
std,2166.682606
min,0.000000
25%,0.000000
50%,9.460000
75%,2619.680000
max,12213.510000


In [151]:
df_list = []
for i, file in enumerate(eolien_files):

    df = pd.read_csv(file, encoding='latin1', sep='\t')
    df_list.append(df)

df_eolien = pd.concat(df_list)

mask = df_eolien['Heure'].isna()
df_eolien=df_eolien.loc[~mask]

df_eolien['Date_Heure_France'] = df_eolien['Date'] + ' ' + df_eolien['Heure']
df_eolien['Date_Heure_France'] = pd.to_datetime(df_eolien['Date_Heure_France'])

df_eolien = df_eolien.reset_index(drop=True)
df_eolien = df_eolien.sort_values('Date_Heure_France', axis = 0)

df_eolien = df_eolien[['Date_Heure_France', 'Prévision J-1', 'Taux de charge J-1']]

date_debut = df_eolien['Date_Heure_France'].loc[0].tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_range = pd.date_range(date_debut, periods = len(df_eolien), freq='1H')
df_eolien.index = date_range.rename('Date_UTC')
#df_eolien = df_eolien.sort_index()
print(df_eolien.columns)
df_eolien.tail()


Index(['Date_Heure_France', 'Prévision J-1', 'Taux de charge J-1'], dtype='object')


,Date_Heure_France,Prévision J-1,Taux de charge J-1
Date_UTC,,,
2022-04-20 05:00:00,2022-12-06 19:00:00,3611.51,20%
2022-04-20 06:00:00,2022-12-06 20:00:00,4043.03,23%
2022-04-20 07:00:00,2022-12-06 21:00:00,4154.14,23%
2022-04-20 08:00:00,2022-12-06 22:00:00,4265.02,24%
2022-04-20 09:00:00,2022-12-06 23:00:00,4376.66,24%
